# Konfiguracja połączenia

import bibliotek

In [1]:
import pandas as pd
import requests
import json
import time
from time import sleep
import datetime
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [2]:
url = 'https://api-datalab.coderslab.com'
airport_end = '/api/v2/airport/{airport_id}'
airport_weather_end = '/api/v2/airportWeather'
aircraft_end = '/api/v2/aircraft'
flight_end = '/api/v2/flight'

APIKEY = "iKRsQ8vdqgT903o2vH1rsejOeQ0F7YC9TvutH6Wk"

headers = {
    'accept': 'application/json',
    'authorization': 'iKRsQ8vdqgT903o2vH1rsejOeQ0F7YC9TvutH6Wk'
}

In [3]:
airports= pd.read_csv(
                    r'..\data\airports.csv', 
                    sep=';', 
                    decimal='.' 
)

airports

,origin_airport_id
0,10874
1,11233
2,13360
3,15008
4,11638
...,...
359,13303
360,13795
361,10693
362,12945


 # Pobieranie `Airport`

In [4]:
request_offset = 60/990
airport_date = []

for airport_id in airports['origin_airport_id']:
    response = requests.get(f'{url}/api/v2/airport/{airport_id}', headers= headers)
    time.sleep(request_offset)
    
    if response.status_code == 200:
        response = json.loads(response.text)
        airport_date.append(response)

    else:
        continue
        print(f"Nie udało się pobrać danych. Kod odpowiedzi: {response.status_code}")
        
airport_df = pd.DataFrame.from_records(airport_date)

In [5]:
airport_df

,ORIGIN_AIRPORT_ID,DISPLAY_AIRPORT_NAME,ORIGIN_CITY_NAME,NAME
0,11638,Fresno Air Terminal,"Fresno, CA","FRESNO YOSEMITE INTERNATIONAL, CA US"
1,13342,General Mitchell Field,"Milwaukee, WI","MILWAUKEE MITCHELL AIRPORT, WI US"
2,13244,Memphis International,"Memphis, TN","MEMPHIS INTERNATIONAL AIRPORT, TN US"
3,15096,Syracuse Hancock International,"Syracuse, NY","SYRACUSE HANCOCK INTERNATIONAL AIRPORT, NY US"
4,10397,Atlanta Municipal,"Atlanta, GA",ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...
...,...,...,...,...
92,13198,Kansas City International,"Kansas City, MO","KANSAS CITY INTERNATIONAL AIRPORT, MO US"
93,10423,Austin - Bergstrom International,"Austin, TX","AUSTIN BERGSTROM INTERNATIONAL AIRPORT, TX US"
94,15370,Tulsa International,"Tulsa, OK","OKLAHOMA CITY WILL ROGERS WORLD AIRPORT, OK US"
95,13303,Miami International,"Miami, FL","MIAMI INTERNATIONAL AIRPORT, FL US"


 Zapis ramki `airport_df` do pliku `airport_list.csv`

In [7]:
airport_df.to_csv(
    r'..\data\raw\airport_list.csv',
    sep=';',
    decimal='.',
    index=False
)

 # Pobieranie `Weather`

In [8]:
endpoint_url = 'https://api-datalab.coderslab.com/api/v2'
request_offset = 60/990
headers = {
    'accept': 'application/json',
    'authorization': 'iKRsQ8vdqgT903o2vH1rsejOeQ0F7YC9TvutH6Wk'
}

start_date = datetime(2019, 1, 1)
end_date = datetime(2020, 3, 31)

weather_data = []

weather_endpoint_url = f"{endpoint_url}/airportWeather"

current_date = start_date
while current_date <= end_date:
    time.sleep(request_offset)
    date_str = current_date.strftime('%Y-%m')
    print(f"Pobieranie danych pogodowych z miesiąca {date_str}")
        
    try:
        r = requests.get(
            f"{weather_endpoint_url}?date={date_str}",
            headers=headers,
        )
            
        if r.status_code == 200:
            response = r.json()
            for record in response:
                weather_data.append(record)
        else:
            print(f"Błąd {r.status_code} przy pobieraniu danych pogodowych z miesiąca {date_str}")
        
    except requests.exceptions.RequestException as e:
        print(f"Błąd połączenia przy pobieraniu danych pogodowych z miesiąca {date_str}: {e}")
        
    current_date += relativedelta(months=1)

weather_df = pd.DataFrame.from_records(weather_data)
airport_weather_df = weather_df

Pobieranie danych pogodowych z miesiąca 2019-01
Pobieranie danych pogodowych z miesiąca 2019-02
Pobieranie danych pogodowych z miesiąca 2019-03
Pobieranie danych pogodowych z miesiąca 2019-04
Pobieranie danych pogodowych z miesiąca 2019-05
Pobieranie danych pogodowych z miesiąca 2019-06
Pobieranie danych pogodowych z miesiąca 2019-07
Pobieranie danych pogodowych z miesiąca 2019-08
Pobieranie danych pogodowych z miesiąca 2019-09
Pobieranie danych pogodowych z miesiąca 2019-10
Pobieranie danych pogodowych z miesiąca 2019-11
Pobieranie danych pogodowych z miesiąca 2019-12
Pobieranie danych pogodowych z miesiąca 2020-01
Pobieranie danych pogodowych z miesiąca 2020-02
Pobieranie danych pogodowych z miesiąca 2020-03


In [9]:
# inna opcja - też działa
"""start_date = datetime(2019, 1, 1)
end_date = datetime(2020, 3, 31)
base_url = 'https://api-datalab.coderslab.com/api/v2/airportWeather'
headers = {
    'accept': 'application/json',
    'authorization': 'iKRsQ8vdqgT903o2vH1rsejOeQ0F7YC9TvutH6Wk'
}

# Inicjalizacja list na dane
dane=[]

# Pobieranie danych dla kolejnych dni
current_date = start_date
while current_date <= end_date:
    formatted_date = current_date.strftime('%Y-%m')
    url = f'{base_url}?date={formatted_date}'
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        dane.append(data[0])

    current_date += relativedelta(months=1)
    
weat_df = pd.DataFrame.from_records(dane)
weat_df"""

"start_date = datetime(2019, 1, 1)\nend_date = datetime(2020, 3, 31)\nbase_url = 'https://api-datalab.coderslab.com/api/v2/airportWeather'\nheaders = {\n    'accept': 'application/json',\n    'authorization': 'iKRsQ8vdqgT903o2vH1rsejOeQ0F7YC9TvutH6Wk'\n}\n\n# Inicjalizacja list na dane\ndane=[]\n\n# Pobieranie danych dla kolejnych dni\ncurrent_date = start_date\nwhile current_date <= end_date:\n    formatted_date = current_date.strftime('%Y-%m')\n    url = f'{base_url}?date={formatted_date}'\n    response = requests.get(url, headers=headers)\n    if response.status_code == 200:\n        data = response.json()\n        dane.append(data[0])\n\n    current_date += relativedelta(months=1)\n    \nweat_df = pd.DataFrame.from_records(dane)\nweat_df"

In [10]:
weather_df

,WT18,STATION,NAME,DATE,AWND,PRCP,SNOW,SNWD,TAVG,TMAX,...,PGTM,WT10,WESD,SN32,SX32,PSUN,TSUN,TOBS,WT07,WT11
0,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-01,4.70,0.14,0.0,0.0,64.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-02,4.92,0.57,0.0,0.0,56.0,59.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-03,5.37,0.15,0.0,0.0,52.0,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-04,12.08,1.44,0.0,0.0,56.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-05,13.42,0.00,0.0,0.0,49.0,59.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46221,None,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",2020-03-27,3.58,0.21,NaN,NaN,NaN,59.0,...,146.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46222,None,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",2020-03-28,6.93,1.29,NaN,NaN,NaN,77.0,...,1535.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46223,None,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",2020-03-29,16.55,0.02,NaN,NaN,NaN,78.0,...,1408.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46224,None,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",2020-03-30,13.42,0.00,NaN,NaN,NaN,57.0,...,817.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Zapis ramki `weather_df` do pliku `airport_weather.csv` w katalogu `data/raw`

In [12]:
weather_df.to_csv(
    r'..\data\raw\airport_weather.csv',
    sep=';',
    decimal='.',
    index=False
)

 # Pobranie `Aircraft`
 #no paramteres

In [13]:
url = 'https://api-datalab.coderslab.com/api/v2/aircraft'
headers = {
    'accept': 'application/json',
    'authorization': 'iKRsQ8vdqgT903o2vH1rsejOeQ0F7YC9TvutH6Wk'
}

response = requests.get(url, headers=headers)
if response.status_code == 200:
    data = response.json()
    
else:
    print(f"Nie udało się pobrać danych. Kod odpowiedzi: {response.status_code}")

aircraft_df = pd.DataFrame.from_records(data)
aircraft_df

,MANUFACTURE_YEAR,TAIL_NUM,NUMBER_OF_SEATS
0,1944,N54514,0.0
1,1945,N1651M,0.0
2,1953,N100CE,0.0
3,1953,N141FL,0.0
4,1953,N151FL,0.0
...,...,...,...
7378,2019,N14011,337.0
7379,2019,N16008,337.0
7380,2019,N16009,337.0
7381,2019,N2250U,276.0


Zapisz ramki `aircraft_df` do pliku `aircraft.csv` w katalogu `data/raw`

In [15]:
aircraft_df.to_csv(
    r'..\data\raw\aircraft.csv',
    sep=';',
    decimal='.',
    index=False
)

 # Pobranie `Flight`

In [16]:
endpoint_url = 'https://api-datalab.coderslab.com/api/v2'
token = 'iKRsQ8vdqgT903o2vH1rsejOeQ0F7YC9TvutH6Wk'
request_offset = 60 / 990  # Limitowanie liczby zapytań
authorization = {'authorization': token}

airports = pd.read_csv(r'..\data\airports.csv', sep=';', decimal='.')
print("Wczytano lotniska")

# Ustawienie dat początkowej i końcowej
start_date = datetime(2019, 1, 1)
end_date = datetime(2020, 3, 31)

flight_data = []

flight_endpoint_url = f"{endpoint_url}/flight"

for airport_id in airports['origin_airport_id']:
    current_date = start_date
    while current_date <= end_date:
        date_str = current_date.strftime('%Y-%m')
        print(f"Pobieranie danych lotów z lotniska {airport_id} z miesiąca {date_str}")

        try:
            r = requests.get(
                f"{flight_endpoint_url}?airportId={airport_id}&date={date_str}",
                headers=authorization,
            )

            if r.status_code == 200:
                response = r.json()
                flight_data.extend(response)
                print(f"Pobrane dane dla lotniska {airport_id} z miesiąca {date_str}")
            elif r.status_code == 204:
                print(f"Brak danych dla lotniska {airport_id} z miesiąca {date_str}")
            else:
                print(f"Błąd {r.status_code} przy pobieraniu danych o lotach z lotniska {airport_id} z miesiąca {date_str}")

        except requests.exceptions.RequestException as e:
            print(f"Błąd połączenia przy pobieraniu danych o lotach z lotniska {airport_id} z miesiąca {date_str}: {e}")

        current_date += relativedelta(months=1)
        time.sleep(request_offset)  
flight_df = pd.DataFrame(flight_data)

Wczytano lotniska
Pobieranie danych lotów z lotniska 10874 z miesiąca 2019-01
Brak danych dla lotniska 10874 z miesiąca 2019-01
Pobieranie danych lotów z lotniska 10874 z miesiąca 2019-02
Brak danych dla lotniska 10874 z miesiąca 2019-02
Pobieranie danych lotów z lotniska 10874 z miesiąca 2019-03
Brak danych dla lotniska 10874 z miesiąca 2019-03
Pobieranie danych lotów z lotniska 10874 z miesiąca 2019-04
Brak danych dla lotniska 10874 z miesiąca 2019-04
Pobieranie danych lotów z lotniska 10874 z miesiąca 2019-05
Brak danych dla lotniska 10874 z miesiąca 2019-05
Pobieranie danych lotów z lotniska 10874 z miesiąca 2019-06
Brak danych dla lotniska 10874 z miesiąca 2019-06
Pobieranie danych lotów z lotniska 10874 z miesiąca 2019-07
Brak danych dla lotniska 10874 z miesiąca 2019-07
Pobieranie danych lotów z lotniska 10874 z miesiąca 2019-08
Brak danych dla lotniska 10874 z miesiąca 2019-08
Pobieranie danych lotów z lotniska 10874 z miesiąca 2019-09
Brak danych dla lotniska 10874 z miesiąca 

Brak danych dla lotniska 11638 z miesiąca 2020-03
Pobieranie danych lotów z lotniska 14150 z miesiąca 2019-01
Brak danych dla lotniska 14150 z miesiąca 2019-01
Pobieranie danych lotów z lotniska 14150 z miesiąca 2019-02
Brak danych dla lotniska 14150 z miesiąca 2019-02
Pobieranie danych lotów z lotniska 14150 z miesiąca 2019-03
Brak danych dla lotniska 14150 z miesiąca 2019-03
Pobieranie danych lotów z lotniska 14150 z miesiąca 2019-04
Brak danych dla lotniska 14150 z miesiąca 2019-04
Pobieranie danych lotów z lotniska 14150 z miesiąca 2019-05
Brak danych dla lotniska 14150 z miesiąca 2019-05
Pobieranie danych lotów z lotniska 14150 z miesiąca 2019-06
Brak danych dla lotniska 14150 z miesiąca 2019-06
Pobieranie danych lotów z lotniska 14150 z miesiąca 2019-07
Brak danych dla lotniska 14150 z miesiąca 2019-07
Pobieranie danych lotów z lotniska 14150 z miesiąca 2019-08
Brak danych dla lotniska 14150 z miesiąca 2019-08
Pobieranie danych lotów z lotniska 14150 z miesiąca 2019-09
Brak danyc

Brak danych dla lotniska 11337 z miesiąca 2020-03
Pobieranie danych lotów z lotniska 13342 z miesiąca 2019-01
Brak danych dla lotniska 13342 z miesiąca 2019-01
Pobieranie danych lotów z lotniska 13342 z miesiąca 2019-02
Brak danych dla lotniska 13342 z miesiąca 2019-02
Pobieranie danych lotów z lotniska 13342 z miesiąca 2019-03
Brak danych dla lotniska 13342 z miesiąca 2019-03
Pobieranie danych lotów z lotniska 13342 z miesiąca 2019-04
Brak danych dla lotniska 13342 z miesiąca 2019-04
Pobieranie danych lotów z lotniska 13342 z miesiąca 2019-05
Brak danych dla lotniska 13342 z miesiąca 2019-05
Pobieranie danych lotów z lotniska 13342 z miesiąca 2019-06
Brak danych dla lotniska 13342 z miesiąca 2019-06
Pobieranie danych lotów z lotniska 13342 z miesiąca 2019-07
Brak danych dla lotniska 13342 z miesiąca 2019-07
Pobieranie danych lotów z lotniska 13342 z miesiąca 2019-08
Brak danych dla lotniska 13342 z miesiąca 2019-08
Pobieranie danych lotów z lotniska 13342 z miesiąca 2019-09
Brak danyc

Brak danych dla lotniska 15096 z miesiąca 2020-03
Pobieranie danych lotów z lotniska 11641 z miesiąca 2019-01
Brak danych dla lotniska 11641 z miesiąca 2019-01
Pobieranie danych lotów z lotniska 11641 z miesiąca 2019-02
Brak danych dla lotniska 11641 z miesiąca 2019-02
Pobieranie danych lotów z lotniska 11641 z miesiąca 2019-03
Brak danych dla lotniska 11641 z miesiąca 2019-03
Pobieranie danych lotów z lotniska 11641 z miesiąca 2019-04
Brak danych dla lotniska 11641 z miesiąca 2019-04
Pobieranie danych lotów z lotniska 11641 z miesiąca 2019-05
Brak danych dla lotniska 11641 z miesiąca 2019-05
Pobieranie danych lotów z lotniska 11641 z miesiąca 2019-06
Brak danych dla lotniska 11641 z miesiąca 2019-06
Pobieranie danych lotów z lotniska 11641 z miesiąca 2019-07
Brak danych dla lotniska 11641 z miesiąca 2019-07
Pobieranie danych lotów z lotniska 11641 z miesiąca 2019-08
Brak danych dla lotniska 11641 z miesiąca 2019-08
Pobieranie danych lotów z lotniska 11641 z miesiąca 2019-09
Brak danyc

Brak danych dla lotniska 15041 z miesiąca 2020-03
Pobieranie danych lotów z lotniska 10529 z miesiąca 2019-01
Brak danych dla lotniska 10529 z miesiąca 2019-01
Pobieranie danych lotów z lotniska 10529 z miesiąca 2019-02
Brak danych dla lotniska 10529 z miesiąca 2019-02
Pobieranie danych lotów z lotniska 10529 z miesiąca 2019-03
Brak danych dla lotniska 10529 z miesiąca 2019-03
Pobieranie danych lotów z lotniska 10529 z miesiąca 2019-04
Brak danych dla lotniska 10529 z miesiąca 2019-04
Pobieranie danych lotów z lotniska 10529 z miesiąca 2019-05
Brak danych dla lotniska 10529 z miesiąca 2019-05
Pobieranie danych lotów z lotniska 10529 z miesiąca 2019-06
Brak danych dla lotniska 10529 z miesiąca 2019-06
Pobieranie danych lotów z lotniska 10529 z miesiąca 2019-07
Brak danych dla lotniska 10529 z miesiąca 2019-07
Pobieranie danych lotów z lotniska 10529 z miesiąca 2019-08
Brak danych dla lotniska 10529 z miesiąca 2019-08
Pobieranie danych lotów z lotniska 10529 z miesiąca 2019-09
Brak danyc

...

Brak danych dla lotniska 10409 z miesiąca 2020-03
Pobieranie danych lotów z lotniska 12519 z miesiąca 2019-01
Brak danych dla lotniska 12519 z miesiąca 2019-01
Pobieranie danych lotów z lotniska 12519 z miesiąca 2019-02
Brak danych dla lotniska 12519 z miesiąca 2019-02
Pobieranie danych lotów z lotniska 12519 z miesiąca 2019-03
Brak danych dla lotniska 12519 z miesiąca 2019-03
Pobieranie danych lotów z lotniska 12519 z miesiąca 2019-04
Brak danych dla lotniska 12519 z miesiąca 2019-04
Pobieranie danych lotów z lotniska 12519 z miesiąca 2019-05
Brak danych dla lotniska 12519 z miesiąca 2019-05
Pobieranie danych lotów z lotniska 12519 z miesiąca 2019-06
Brak danych dla lotniska 12519 z miesiąca 2019-06
Pobieranie danych lotów z lotniska 12519 z miesiąca 2019-07
Brak danych dla lotniska 12519 z miesiąca 2019-07
Pobieranie danych lotów z lotniska 12519 z miesiąca 2019-08
Brak danych dla lotniska 12519 z miesiąca 2019-08
Pobieranie danych lotów z lotniska 12519 z miesiąca 2019-09
Brak danyc

Brak danych dla lotniska 10779 z miesiąca 2020-03
Pobieranie danych lotów z lotniska 11898 z miesiąca 2019-01
Brak danych dla lotniska 11898 z miesiąca 2019-01
Pobieranie danych lotów z lotniska 11898 z miesiąca 2019-02
Brak danych dla lotniska 11898 z miesiąca 2019-02
Pobieranie danych lotów z lotniska 11898 z miesiąca 2019-03
Brak danych dla lotniska 11898 z miesiąca 2019-03
Pobieranie danych lotów z lotniska 11898 z miesiąca 2019-04
Brak danych dla lotniska 11898 z miesiąca 2019-04
Pobieranie danych lotów z lotniska 11898 z miesiąca 2019-05
Brak danych dla lotniska 11898 z miesiąca 2019-05
Pobieranie danych lotów z lotniska 11898 z miesiąca 2019-06
Brak danych dla lotniska 11898 z miesiąca 2019-06
Pobieranie danych lotów z lotniska 11898 z miesiąca 2019-07
Brak danych dla lotniska 11898 z miesiąca 2019-07
Pobieranie danych lotów z lotniska 11898 z miesiąca 2019-08
Brak danych dla lotniska 11898 z miesiąca 2019-08
Pobieranie danych lotów z lotniska 11898 z miesiąca 2019-09
Brak danyc

Brak danych dla lotniska 10620 z miesiąca 2020-03
Pobieranie danych lotów z lotniska 14222 z miesiąca 2019-01
Brak danych dla lotniska 14222 z miesiąca 2019-01
Pobieranie danych lotów z lotniska 14222 z miesiąca 2019-02
Brak danych dla lotniska 14222 z miesiąca 2019-02
Pobieranie danych lotów z lotniska 14222 z miesiąca 2019-03
Brak danych dla lotniska 14222 z miesiąca 2019-03
Pobieranie danych lotów z lotniska 14222 z miesiąca 2019-04
Brak danych dla lotniska 14222 z miesiąca 2019-04
Pobieranie danych lotów z lotniska 14222 z miesiąca 2019-05
Brak danych dla lotniska 14222 z miesiąca 2019-05
Pobieranie danych lotów z lotniska 14222 z miesiąca 2019-06
Brak danych dla lotniska 14222 z miesiąca 2019-06
Pobieranie danych lotów z lotniska 14222 z miesiąca 2019-07
Brak danych dla lotniska 14222 z miesiąca 2019-07
Pobieranie danych lotów z lotniska 14222 z miesiąca 2019-08
Brak danych dla lotniska 14222 z miesiąca 2019-08
Pobieranie danych lotów z lotniska 14222 z miesiąca 2019-09
Brak danyc

Brak danych dla lotniska 15454 z miesiąca 2020-03
Pobieranie danych lotów z lotniska 15370 z miesiąca 2019-01
Brak danych dla lotniska 15370 z miesiąca 2019-01
Pobieranie danych lotów z lotniska 15370 z miesiąca 2019-02
Brak danych dla lotniska 15370 z miesiąca 2019-02
Pobieranie danych lotów z lotniska 15370 z miesiąca 2019-03
Brak danych dla lotniska 15370 z miesiąca 2019-03
Pobieranie danych lotów z lotniska 15370 z miesiąca 2019-04
Brak danych dla lotniska 15370 z miesiąca 2019-04
Pobieranie danych lotów z lotniska 15370 z miesiąca 2019-05
Brak danych dla lotniska 15370 z miesiąca 2019-05
Pobieranie danych lotów z lotniska 15370 z miesiąca 2019-06
Brak danych dla lotniska 15370 z miesiąca 2019-06
Pobieranie danych lotów z lotniska 15370 z miesiąca 2019-07
Brak danych dla lotniska 15370 z miesiąca 2019-07
Pobieranie danych lotów z lotniska 15370 z miesiąca 2019-08
Brak danych dla lotniska 15370 z miesiąca 2019-08
Pobieranie danych lotów z lotniska 15370 z miesiąca 2019-09
Brak danyc

Pobrane dane dla lotniska 13303 z miesiąca 2020-03
Pobieranie danych lotów z lotniska 13795 z miesiąca 2019-01
Brak danych dla lotniska 13795 z miesiąca 2019-01
Pobieranie danych lotów z lotniska 13795 z miesiąca 2019-02
Brak danych dla lotniska 13795 z miesiąca 2019-02
Pobieranie danych lotów z lotniska 13795 z miesiąca 2019-03
Brak danych dla lotniska 13795 z miesiąca 2019-03
Pobieranie danych lotów z lotniska 13795 z miesiąca 2019-04
Brak danych dla lotniska 13795 z miesiąca 2019-04
Pobieranie danych lotów z lotniska 13795 z miesiąca 2019-05
Brak danych dla lotniska 13795 z miesiąca 2019-05
Pobieranie danych lotów z lotniska 13795 z miesiąca 2019-06
Brak danych dla lotniska 13795 z miesiąca 2019-06
Pobieranie danych lotów z lotniska 13795 z miesiąca 2019-07
Brak danych dla lotniska 13795 z miesiąca 2019-07
Pobieranie danych lotów z lotniska 13795 z miesiąca 2019-08
Brak danych dla lotniska 13795 z miesiąca 2019-08
Pobieranie danych lotów z lotniska 13795 z miesiąca 2019-09
Brak dany

In [17]:
flight_df

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,CRS_DEP_TIME,DEP_TIME,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,YEAR,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,1,20,7,WN,N204WN,682,10397,11292,605,602.0,...,205,204.0,1199,5,2019,NaN,NaN,NaN,NaN,NaN
1,1,20,7,WN,N8682B,2622,10397,11292,2120,2114.0,...,210,205.0,1199,5,2019,NaN,NaN,NaN,NaN,NaN
2,1,20,7,WN,N717SA,2939,10397,11292,1800,1807.0,...,210,220.0,1199,5,2019,4.0,0.0,10.0,0.0,3.0
3,1,20,7,WN,N709SW,3848,10397,11292,1355,1354.0,...,205,204.0,1199,5,2019,NaN,NaN,NaN,NaN,NaN
4,1,20,7,WN,N7864B,1352,10397,11697,1125,1125.0,...,120,124.0,581,3,2019,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1386115,3,26,4,DL,N350DN,1982,13303,12953,1956,1944.0,...,183,169.0,1096,5,2020,NaN,NaN,NaN,NaN,NaN
1386116,3,26,4,DL,N908DE,1987,13303,10397,1120,1117.0,...,121,109.0,594,3,2020,NaN,NaN,NaN,NaN,NaN
1386117,3,26,4,DL,NaN,1998,13303,10397,1817,NaN,...,125,NaN,594,3,2020,NaN,NaN,NaN,NaN,NaN
1386118,3,26,4,DL,N352NW,2025,13303,10397,1937,1928.0,...,123,107.0,594,3,2020,NaN,NaN,NaN,NaN,NaN


Zapis ramki `flight_df` do pliku `flight.csv` w katalogu `data/raw`

In [19]:
flight_df.to_csv(
    r'..\data\raw\flight.csv',
    sep=';',
    decimal='.',
    index=False
)